In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)
__logger__ = logging.getLogger(__name__)

In [ ]:
from tsdm.types.protocols import *

In [ ]:
class Foo:
    pass

In [ ]:
from typing import *

In [ ]:
# assert issubclass(NTuple, tuple)
assert not issubclass(tuple, NTuple)

assert isinstance((1, 2), tuple)
assert not isinstance((1, 2), NTuple)

In [ ]:
assert issubclass(NTuple, tuple)

In [ ]:
class FooMeta(type):
    def __subclasscheck__(cls, subclass):
        print(cls, subclass)
        return True

In [ ]:
class Foo(metaclass=FooMeta):
    ...

In [ ]:
issubclass(Foo, tuple)

In [ ]:
issubclass(tuple, Foo)

In [ ]:
assert issubclass(Point, NTuple)

In [ ]:
# test sample NamedTuple class
class Point(NamedTuple):
    x: float
    y: float


assert issubclass(Point, NTuple)
# assert issubclass(Point, tuple)
# assert not issubclass(tuple, Point)
# assert not issubclass(NTuple, Point)

In [ ]:
assert isinstance(Point(1, 2), NTuple)
assert not isinstance((1, 2), Point)

In [ ]:
issubclass(Foo, NTuple), isinstance(Foo(), NTuple), issubclass(Foo, tuple), issubclass(
    NTuple, tuple
), issubclass(Foo, tuple)

In [ ]:
from collections.abc import Iterator, Mapping, Sequence
from dataclasses import Field
from typing import Any, NamedTuple, Protocol, TypeVar, overload, runtime_checkable

from typing_extensions import Self, SupportsIndex, get_original_bases

from tsdm.types.variables import any_co as T_co
from tsdm.types.variables import any_var as T
from tsdm.types.variables import key_contra, key_var, value_co

scalar_co = TypeVar("scalar_co", covariant=True)
# Either: TypeAlias = Union[T, "Array[T]"]
A = TypeVar("A", bound="Array")


class SelfMap(Protocol):
    r"""A callback for endofunctions."""

    def __call__(self, __x: T) -> T:
        r"""Returns the result of the endofunction."""


class SelfMapProto(Protocol[T]):
    r"""A generic protocol for endofunctions."""

    def __call__(self, __x: T) -> T:
        r"""Returns the result of the endofunction."""


@runtime_checkable
class Dataclass(Protocol):
    r"""Protocol for anonymous dataclasses."""

    @property
    def __dataclass_fields__(self) -> Mapping[str, Field]:
        r"""Return the fields of the dataclass."""

In [ ]:
class NamedTupleMeta(type(Protocol)):
    def __subclasscheck__(cls, subclass):
        # implements issubclass(subclass, cls)
        # if issubclass
        print(f"__subclasscheck__ called {cls=} {subclass=}")
        return True


@runtime_checkable
class NTuple(Protocol[T_co], metaclass=NamedTupleMeta):
    r"""Protocol for anonymous namedtuple.

    References:
        - https://github.com/python/typeshed/blob/main/stdlib/builtins.pyi
    """

    # @classmethod
    # def __subclasshook__(cls, other: type) -> bool:
    #     """Cf https://github.com/python/cpython/issues/106363."""
    #     print(f"__subclasshook__ called !{cls=}, {other=}")
    #     if cls is tuple:
    #         return True
    #     if NamedTuple in get_original_bases(other):
    #         return True
    #     return NotImplemented

    # fmt: off
    @property
    def _fields(self) -> tuple[str, ...]: ...

    def _asdict(self) -> Mapping[str, T_co]: ...
    # def __new__(cls, __iterable: Iterable[T_co] = ...) -> Self: ...
    def __len__(self) -> int: ...
    def __contains__(self, __key: object) -> bool: ...
    @overload
    def __getitem__(self, __key: SupportsIndex) -> T_co: ...
    @overload
    def __getitem__(self, __key: slice) -> tuple[T_co, ...]: ...
    def __iter__(self) -> Iterator[T_co]: ...
    # def __lt__(self, __value: tuple[T_co, ...]) -> bool: ...
    # def __le__(self, __value: tuple[T_co, ...]) -> bool: ...
    # def __gt__(self, __value: tuple[T_co, ...]) -> bool: ...
    # def __ge__(self, __value: tuple[T_co, ...]) -> bool: ...
    # @overload
    # def __add__(self, __value: tuple[T_co, ...]) -> tuple[T_co, ...]: ...
    # @overload
    # def __add__(self, __value: tuple[T, ...]) -> tuple[T_co | T, ...]: ...
    # def __mul__(self, __value: SupportsIndex) -> tuple[T_co, ...]: ...
    # def __rmul__(self, __value: SupportsIndex) -> tuple[T_co, ...]: ...
    # def count(self, __value: Any) -> int: ...
    # def index(self, __value: Any, __start: SupportsIndex = 0, __stop: SupportsIndex = sys.maxsize) -> int: ...
    # fmt: on

In [ ]:
issubclass(NTuple, tuple)

In [ ]:
assert not issubclass(tuple, NTuple)

assert isinstance((1, 2), tuple)
assert not isinstance((1, 2), NTuple)


# test sample NamedTuple class
class Point(NamedTuple):
    x: float
    y: float


assert issubclass(Point, NTuple)
assert issubclass(Point, tuple)
assert not issubclass(tuple, Point)
assert not issubclass(NTuple, Point)

assert isinstance(Point(1, 2), NTuple)
assert not isinstance((1, 2), Point)

In [ ]:
type(Protocol)

In [ ]:
from typing import NamedTuple

from typing_extensions import get_original_bases


def is_namedtuple_cls(cls):
    return NamedTuple in get_original_bases(cls)
    # return any(NamedTuple in get_original_bases(supercls) for supercls in cls.__mro__)


# def is_namedtuple_cls(cls):
#     return issubclass(cls, tuple) and all(
#         hasattr(cls, attr) for attr in ("_fields", "_asdict", "_make", "_replace")
#     )


from typing import Any, Protocol, runtime_checkable


@runtime_checkable
class NamedTupleProto(Protocol):
    def _asdict(self) -> dict[str, Any]:
        ...

    @classmethod
    def __subclasshook__(cls, other: type) -> bool:
        if NamedTuple in get_original_bases(other):
            return True
        # if cls is NamedTupleProto and is_namedtuple_cls(other):
        #     return True
        return NotImplemented


class Point(NamedTuple):
    x: float
    y: float


class Bar:
    """..."""

    _asdict = None


isinstance(Point(0, 0), NamedTupleProto), issubclass(
    Point, NamedTupleProto
), isinstance(Bar(), NamedTupleProto), issubclass(Bar, NamedTupleProto)

In [ ]:
import pandas as pd
import pyarrow as pa

df = pd.DataFrame(
    {
        "n_legs": [None, 4, 5, 100],
        "animals": ["Flamingo", "Horse", "Brittle stars", "Centipede"],
    }
)
table = pa.Table.from_pandas(df)
x = table.to_pydict()

In [ ]:
df.reset_index().to_dict()

In [ ]:
df.animals.to_dict()

In [ ]:
arr = table["animals"]

In [ ]:
arr.to_pylist()

In [ ]:
from collections import namedtuple

In [ ]:
hash(tuple((key, tuple(values)) for key, values in table.to_pydict().items()))

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import torch

In [ ]:
import logging
import pickle

import numpy as np
import pandas as pd
import pytest
import torch
from pandas import DataFrame

from tsdm.encoders import (
    BaseEncoder,
    BoundaryEncoder,
    BoxCoxEncoder,
    FastFrameEncoder,
    FrameAsDict,
    IdentityEncoder,
    LogitBoxCoxEncoder,
    MinMaxScaler,
    StandardScaler,
    TimeDeltaEncoder,
)
from tsdm.tasks import KiwiTask

logging.basicConfig(level=logging.INFO)
__logger__ = logging.getLogger(__name__)

SplitID = (0, "train")

task = KiwiTask()
ts = task.dataset.timeseries.iloc[:20_000]  # use first 20_000 values only
VF = task.dataset.timeseries_description
sampler = task.samplers[SplitID]
generator = task.generators[SplitID]
key = next(iter(sampler))
sample = generator[key]
x = sample.inputs.x

In [ ]:
# Construct the encoder
column_encoders: dict[str, BaseEncoder] = {}
for col, scale, lower, upper in VF[["scale", "lower", "upper"]].itertuples():
    match scale:
        case "percent":
            column_encoders[col] = (
                LogitBoxCoxEncoder()
                @ MinMaxScaler(0, 1, xmin=lower, xmax=upper)
                @ BoundaryEncoder(lower, upper, mode="clip")
            )
        case "absolute":
            if upper < np.inf:
                column_encoders[col] = (
                    BoxCoxEncoder()
                    # @ MinMaxScaler(lower, upper)
                    @ BoundaryEncoder(lower, upper, mode="clip")
                )
            else:
                column_encoders[col] = BoxCoxEncoder() @ BoundaryEncoder(
                    lower, upper, mode="clip"
                )
        case "linear":
            column_encoders[col] = IdentityEncoder()
        case _:
            raise ValueError(f"{scale=} unknown")

encoder = (
    FrameAsDict(
        groups={
            "key": ["run_id", "experiment_id"],
            "T": ["measurement_time"],
            "X": ...,
        },
        dtypes={"T": "float32", "X": "float32"},
        encode_index=True,
    )
    @ StandardScaler(axis=-1)
    @ FastFrameEncoder(
        column_encoders=column_encoders,
        index_encoders={"measurement_time": MinMaxScaler() @ TimeDeltaEncoder()},
    )
);

In [ ]:
encoder.fit(ts)  # fit encoder to the whole dataset
encoded = encoder.encode(ts)
decoded = encoder.decode(encoded)
MAD = (decoded - ts).abs().mean().mean()
assert all(decoded.isna() == ts.isna()), "NaN pattern mismatch"
assert MAD < 1e-3, "Large deviations from original values"
# check that the encoded values are within the bounds

In [ ]:
# apply encoder to a single slice
encoded = encoder.encode(x)
xhat = DataFrame(encoded["X"])
assert (xhat.isna().values == x.isna().values).all(), "NaN pattern mismatch"

In [ ]:
enc = encoder[-1]["Acetate"][-1]
enc.encode(ts.Acetate)

In [ ]:
encoded

In [ ]:
# check that decoded matches with original
decoded = encoder.decode(encoded)
MAD = (decoded - x).abs().mean().mean()
assert all(decoded.isna() == x.isna()), "NaN pattern mismatch"
assert MAD < 1e-3, "Large deviations from original values"

# check that decoding random values satisfies bounds
rng_data = torch.randn_like(encoded["X"])
encoded["X"] = 20 * rng_data  # large std. dev. to ensure that the bounds are violated
decoded = encoder.decode(encoded)
bounds = pd.concat([decoded.min(), decoded.max()], axis=1, keys=["lower", "upper"])
for col, lower, upper in bounds.itertuples():
    match VF.loc[col, "scale"]:
        case "percent":
            assert lower == 0, "Lower bound violated"
            assert upper == 100, "Upper bound violated"
        case "absolute":
            assert lower == 0, "Lower bound violated"

# test_serialization
with open("trained_encoder.pickle", "wb") as file:
    pickle.dump(encoder, file)

with open("trained_encoder.pickle", "rb") as file:
    loaded_encoder = pickle.load(file)

assert isinstance(loaded_encoder, BaseEncoder)